In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

## Do Cleaning

In [ ]:
mktcap = pd.read_pickle("clean_mktcap.pickle")
price  = pd.read_pickle("clean_price.pickle")

### 1. BTC의 weights 기준으로 클리닝

In [ ]:
# Mkt index에서 BTC의 비율을 시각화 해보자
weight = mktcap.apply(lambda x: x / np.nansum(x), axis=1)
btc = weight[["BTC"]]
btc.plot(figsize=(24,12));

In [ ]:
# BTC의 비중이 갑자기 많이 변하는 날들... 한번 확인해볼 필요가 있다
pct = btc.pct_change(fill_method=None)
tmp_problem = btc.loc[pct.abs().values > 0.1]
tmp_problem

In [ ]:
# 2013-12-26 ~ 2013-12-28 은 여전히 문제가 있다(BTC의 비중이 갑자기 튀는 현상)
btc.loc["2013-12-26":"2013-12-28"]

In [ ]:
# 13년 12월 26일 -> 2013년 12월 27일 사이에 무슨 코인이 문제인지를 찾아보자
sample = weight.loc["2013-12-26":"2013-12-27"]
pct = sample.pct_change(fill_method=None)
pct.iloc[-1].sort_values(ascending=False).dropna()
# MEC, QRK, ANC가 이상하다 직접 찾아보자

In [ ]:
# MEC 점검
fig, ax = plt.subplots(1,1)
price[["MEC"]].loc["2013-12-25":"2013-12-31"] \
              .plot(figsize=(24,12), ax=ax, 
                    sharex=True, alpha=1, c="k")
mktcap[["MEC"]].loc["2013-12-25":"2013-12-31"] \
               .plot(figsize=(24,12), ax=ax,
                     sharex=True, secondary_y = True, alpha=0.5);
# price의 상승대비 mktcap이 튀어오르는게 확실히 문제가 있다(2014년)

mktcap[["MEC"]].loc["2013-12-25":"2013-12-31"]
# 12월 27일에 시총이 튄다 -> 이거는 잘못된 데이터인 것 같음, 이후에도 가격은 계속 비슷하기 때문임
## Coinmarketcap에서는 트래킹 할 수 없는 코인이라고 뜸, 다른 벤더도 마찬가지

### Drop 하는게 맞을듯
price.drop("MEC", axis=1, inplace=True)
mktcap.drop("MEC", axis=1, inplace=True)

In [ ]:
# QRK 점검
fig, ax = plt.subplots(1,1)
price[["QRK"]].loc["2013-12-25":"2013-12-31"] \
              .plot(figsize=(8,4), ax=ax, 
                    sharex=True, alpha=1, c="k")
mktcap[["QRK"]].loc["2013-12-25":"2013-12-31"] \
               .plot(figsize=(8,4), ax=ax,
                     sharex=True, secondary_y = True, alpha=0.5);

mktcap[["QRK"]].loc["2013-12-25":"2013-12-31"]
# 27일 날 갑자기 cap이 튄다 -> Coinmarketcap 확인결과 잘못된 데이터가 맞다
## price로 smoothing 해줘야 할듯

In [ ]:
# QRK 스무딩
price_0 = price[["QRK"]].loc["2013-12-26"].values[0]
price_1 = price[["QRK"]].loc["2013-12-27"].values[0]
print("가격: {} -> {}".format(price_0, price_1))

mktcap_0 = mktcap[["QRK"]].loc["2013-12-26"].values[0]
mktcap_1 = mktcap[["QRK"]].loc["2013-12-27"].values[0]
print("시총: {} -> {}".format(mktcap_0, mktcap_1))

pct = (price_1 - price_0) / price_0
new_mktcap = mktcap_0 * (1+pct)
print("새로운 시총:" , new_mktcap)
mktcap["QRK"].loc["2013-12-27"] = new_mktcap

In [ ]:
# ANC 점검
fig, ax = plt.subplots(1,1)
price[["ANC"]].loc["2013-12-25":"2013-12-31"] \
              .plot(figsize=(8,4), ax=ax, 
                    sharex=True, alpha=1, c="k")
mktcap[["ANC"]].loc["2013-12-25":"2013-12-31"] \
               .plot(figsize=(8,4), ax=ax,
                     sharex=True, secondary_y = True, alpha=0.5);

mktcap[["ANC"]].loc["2013-12-25":"2013-12-31"]

# 똑같이 price로 smoothing 하면 됨

In [ ]:
# ANC 스무딩
price_0 = price[["ANC"]].loc["2013-12-26"].values[0]
price_1 = price[["ANC"]].loc["2013-12-27"].values[0]
print("가격: {} -> {}".format(price_0, price_1))

mktcap_0 = mktcap[["ANC"]].loc["2013-12-26"].values[0]
mktcap_1 = mktcap[["ANC"]].loc["2013-12-27"].values[0]
print("시총: {} -> {}".format(mktcap_0, mktcap_1))

pct = (price_1 - price_0) / price_0
new_mktcap = mktcap_0 * (1+pct)
mktcap["ANC"].loc["2013-12-27"] = new_mktcap

### 2. 중간중간 발견한 이상한 코인들 점검

In [ ]:
# NXT 점검
fig, ax = plt.subplots(1,1)
price[["NXT"]].loc["2013-12":"2014-01"] \
              .plot(figsize=(8,4), ax=ax, sharex=True,
                    alpha=1, c="k")
              
mktcap[["NXT"]].loc["2013-12":"2014-01"] \
               .plot(figsize=(8,4), ax=ax, sharex=True, 
                     secondary_y = True, alpha=0.5);
mktcap[["NXT"]].loc["2013-12-25":"2013-12-31"]
# coinmarketcap과 비교결과 잘못된 데이터가 맞다는 것을 확인
## price를 통해 smothing 해주도록 하겠음

In [ ]:
# NXT 스무딩
price_0 = price[["NXT"]].loc["2013-12-27"].values[0]
price_1 = price[["NXT"]].loc["2013-12-28"].values[0]
print("가격: {} -> {}".format(price_0, price_1))

mktcap_0 = mktcap[["NXT"]].loc["2013-12-27"].values[0]
mktcap_1 = mktcap[["NXT"]].loc["2013-12-28"].values[0]
print("시총: {} -> {}".format(mktcap_0, mktcap_1))

pct = (price_1 - price_0) / price_0
new_mktcap = mktcap_0 * (1+pct)
mktcap["NXT"].loc["2013-12-28"] = new_mktcap

In [ ]:
# RISE2
fig, ax = plt.subplots(1,1)
price[["RISE2"]].plot(figsize=(24,12), ax=ax, sharex=True,
                    alpha=1, c='k')
mktcap[["RISE2"]].plot(figsize=(24,12), ax=ax, sharex=True,
                     secondary_y = True,
                     alpha=0.5);

# 가격이 e-08에서 하루만에 e-04가 됨
price[["RISE2"]].loc[~price["RISE2"].isna()].loc["2021-11-28":"2021-12-05"]

In [ ]:
# RISE2는 Coinmarketcap에서도 22년 1월 24일부터 데이터가 있어서 우리도 그 sample period기간만 사용하면 될듯!!
price["RISE2"].loc[:"2022-01-23"] = np.nan
mktcap["RISE2"].loc[:"2022-01-23"] = np.nan

# RISE2 -> 잘 제거되었는지 확인
fig, ax = plt.subplots(1,1)
price[["RISE2"]].plot(figsize=(24,12), ax=ax, sharex=True,
                    alpha=1, c='k')
mktcap[["RISE2"]].plot(figsize=(24,12), ax=ax, sharex=True,
                     secondary_y = True,
                     alpha=0.5);

In [ ]:
mktcap_pct = mktcap.pct_change(fill_method=None)
price_pct = price.pct_change(fill_method=None)

In [ ]:
# cap의 pct_change가 daily 10_000%를 넘어가는 코인이 있다?!!
print("문제있는 데이터의 수:", (~mktcap_pct[np.abs(mktcap_pct) > 10].isna()).sum().sum())

# 무슨 코인들이 해당하는가
coin = (~mktcap_pct[mktcap_pct > 10].isna()).sum()
coin_lst = coin.loc[coin >=1].index.tolist()

In [ ]:
# Plot으로 확인해보자
for ticker in coin_lst:
    fig, ax = plt.subplots(1,1)
    price[ticker].plot(figsize=(24,12), ax=ax, sharex=True,
                        alpha=1, c="k")
    mktcap[ticker].plot(figsize=(24,12), ax=ax, sharex=True, 
                         secondary_y = True, alpha=0.5, c='r')
    plt.legend(["price", "mktcap"])
    plt.title(ticker);

In [ ]:
# 확인한놈들 몇개는 그냥 드랍시키자

price.drop(["AUCTION", "BCA", "BONE2", "BONO", "BTCST", 
            "CGT2", "CRD", "CRED", "CVCOIN", "DG", "IFC", 
            "KARMA2", "LDOGE", "MOON", "OUSD", "REV"], axis=1, 
           inplace=True)
mktcap.drop(["AUCTION", "BCA", "BONE2", "BONO", "BTCST", 
            "CGT2", "CRD", "CRED", "CVCOIN", "DG", "IFC", 
            "KARMA2", "LDOGE", "MOON", "OUSD", "REV"], axis=1, 
           inplace=True)

In [ ]:
# 저장
mktcap.to_pickle("jehyeon_cleaning_mktcap.pickle")
price.to_pickle("jehyeon_cleaning_price.pickle")

## Check after cleaning

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
price = pd.read_pickle("jehyeon_cleaning_price.pickle")
mktcap = pd.read_pickle("jehyeon_cleaning_mktcap.pickle")

In [ ]:
weight = mktcap.apply(lambda x: x / np.nansum(x), axis=1)
btc = weight[["BTC"]]
btc.plot(figsize=(24,12));